In [41]:
import pandas as pd
import re
import os

os.makedirs("data_clean", exist_ok=True)

# ========================= File path =========================
aset_file_path = "/content/Aset Ritel Jatim.csv"
user_file_path = "/content/data_user.csv"

# ========================= Load data =========================
aset_data = pd.read_csv(aset_file_path)
user_data = pd.read_csv(user_file_path)

# ========================= Drop duplicates =========================
aset_data = aset_data.drop_duplicates(subset=["FATID"], keep="first")
user_data = user_data.drop_duplicates(subset=["ID Permohonan"], keep="first")

# ========================= Konversi tipe data agar sesuai SQL =========================
aset_data = aset_data.astype({
    # Kolom dari Tabel OLT
    "Hostname OLT": "string",
    "Latitude OLT": "string",
    "Longtitude OLT": "string",
    "Brand OLT": "string",
    "Type OLT": "string",
    "Kapasitas OLT": "Int64",
    "Kapasitas port OLT": "Int64",
    "OLT Port": "Int64",
    "OLT": "string",
    "Interface OLT": "string",

    # Kolom dari Tabel FDT
    "FDTID": "string",
    "Status OSP AMARTA FDT": "string",
    "Jumlah Splitter FDT": "Int64",
    "Kapasitas Splitter FDT": "Int64",
    "FDT New/Existing": "string",
    "Port FDT": "Int64",
    "Latitude FDT": "string",
    "Longtitude FDT": "string",

    # Kolom dari Tabel FAT
    "FATID": "string",
    "Jumlah Splitter FAT": "Int64",
    "Kapasitas Splitter FAT": "Int64",
    "Latitude FAT": "string",
    "Longtitude FAT": "string",
    "Status OSP AMARTA FAT": "string",
    "FAT KONDISI": "string",
    "FAT FILTER PEMAKAIAN": "string",
    "KETERANGAN FULL": "string",
    "FAT ID X": "string",
    "FILTER FAT CAP": "string",

    # Kolom dari Tabel Cluster
    "Latitude Cluster": "string",
    "Longtitude Cluster": "string",
    "Area KP": "string",
    "Kota Kab": "string",
    "Kecamatan": "string",
    "Kelurahan": "string",
    "UP3": "string",
    "ULP": "string",

    # Kolom dari Tabel Dokumentasi
    "LINK DOKUMEN FEEDER": "string",
    "KETERANGAN DOKUMEN": "string",
    "LINK DATA ASET": "string",
    "KETERANGAN DATA ASET": "string",
    "LINK MAPS": "string",
    "UPDATE ASET": "string",
    "AMARTA UPDATE": "string",

    # Kolom dari Tabel HomeConnected
    "HC OLD": "Int64",
    "HC iCRM+": "Int64",
    "TOTAL HC": "Int64",
    "CLEANSING HP": "string",

    # Kolom dari Tabel AdditionalInformation
    "PA": "string",
    "Tanggal RFS": "string",
    "Mitra": "string",
    "Kategori": "string",
    "Sumber Datek": "string"
})


# Konversi tipe data user_data (tabel Pelanggan)
user_data = user_data.astype({
    "SID": "string",
    "ID Permohonan": "string",
    "Latitude Pelanggan": "string",
    "Longtitude Pelanggan": "string",
    "Cust Name": "string",
    "Telpn": "string",
    "ID FAT": "string",
    "Latitude FAT": "string",
    "Longtitude FAT": "string",
    "Hostname OLT": "string",
    "FDT": "string",
    "NOTES": "string"
})

# ========================= Cek hasil =========================
print("📋 Info aset_data setelah konversi:")
print(aset_data.info())
print("📋 Info user_data setelah konversi:")
print(user_data.info())
print("✅ Data duplikat telah dihapus dan tipe data dikonversi sesuai SQL schema.")

<ipython-input-41-f5ba98d2a069>:12: DtypeWarning: Columns (9,21,23,39) have mixed types. Specify dtype option on import or set low_memory=False.
  aset_data = pd.read_csv(aset_file_path)
<ipython-input-41-f5ba98d2a069>:13: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  user_data = pd.read_csv(user_file_path)


📋 Info aset_data setelah konversi:
<class 'pandas.core.frame.DataFrame'>
Index: 30818 entries, 0 to 30850
Data columns (total 55 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   PA                      29316 non-null  string
 1   Tanggal RFS             16271 non-null  string
 2   Mitra                   25155 non-null  string
 3   Kategori                30587 non-null  string
 4   Area KP                 30818 non-null  string
 5   Kota Kab                30700 non-null  string
 6   Lokasi OLT              30272 non-null  object
 7   Hostname OLT            30759 non-null  string
 8   Latitude OLT            30627 non-null  string
 9   Longtitude OLT          28860 non-null  string
 10  Brand OLT               30601 non-null  string
 11  Type OLT                30490 non-null  string
 12  Kapasitas OLT           28981 non-null  Int64 
 13  Kapasitas port OLT      19173 non-null  Int64 
 14  OLT Port                

In [42]:
import pandas as pd
import re

# ==================== Fungsi Pembersih ====================
def clean_degree_with_comma(coord):
    if pd.isna(coord) or 'Â,' not in str(coord):
        return None
    coord = str(coord).replace('Â', '')
    return coord

def clean_degree_separated(coord):
    if pd.isna(coord) or 'Â°' not in str(coord):
        return None
    first_deg_index = str(coord).find('Â°')
    coord = str(coord)[:first_deg_index + 2] + "," + str(coord)[first_deg_index + 2:]
    coord = coord.replace('Â°', '')
    return coord

def clean_degree_as_separator(coord):
    if pd.isna(coord) or '°' not in str(coord):
        return None
    first_deg_index = str(coord).find('°')
    coord = str(coord)[:first_deg_index] + "," + str(coord)[first_deg_index + 1:]
    coord = coord.replace('°', '')
    return coord

def clean_two_commas_with_space(coord):
    if pd.isna(coord) or ' ' not in str(coord) or str(coord).count(',') != 2:
        return None
    try:
        lat, lon = str(coord).split()
        lat = lat.replace(',', '.')
        lon = lon.replace(',', '.')
        return f"{lat},{lon}"
    except:
        return None

def clean_comma_separated(coord):
    if pd.isna(coord):
        return None
    parts = [part.strip() for part in str(coord).split(',')]
    if len(parts) == 2:
        lat = parts[0].replace(',', '.')
        lon = parts[1].replace(',', '.')
        return f"{lat},{lon}"
    return None

def clean_merged_coordinates(coord):
    if pd.isna(coord):
        return None
    coord = str(coord).replace(" ", "").replace(",", "")
    match = re.match(r'^(-?\d+\.\d+)(\d{3}\.\d+)$', coord)
    if match:
        lat, lon = match.groups()
        return f"{lat},{lon}"
    return None

def move_comma_separator(coord):
    if pd.isna(coord) or ',' not in str(coord) or str(coord).count('.') != 2:
        return None
    try:
        lat_part, lon_part = str(coord).split(',')
        lat_before, lat_after = lat_part.split('.')
        lon_before, lon_after = lon_part.split('.')
        if len(lon_before) != 3:
            return None
        lat_after = lat_after[:6]
        new_lon = lat_after[6:] + lon_part
        lon_before, lon_after = new_lon.split('.')
        lon_before = lon_before.zfill(3)
        new_lat = f"{lat_before}.{lat_after[:6]}"
        new_lon = f"{lon_before}.{lon_after}"
        return f"{new_lat},{new_lon}"
    except:
        return None

def clean_dot_separated_no_comma(coord):
    if pd.isna(coord):
        return None
    coord = str(coord).strip()

    # Cek apakah ada dua titik dan tidak ada koma
    if coord.count('.') == 3 or (coord.count('.') == 2 and ',' not in coord):
        try:
            # Coba ekstrak dua nilai float dari string
            match = re.match(r'^(-?\d+\.\d+)\.(\d+\.\d+)$', coord)
            if match:
                lat = match.group(1)
                lon = match.group(2)
                return f"{lat},{lon}"
        except:
            return None
    return None

# ==================== Fungsi Utama Pembersih ====================
def apply_cleaning_debug(coord):
    if pd.isna(coord):
        return None, "NaN"
    coord = str(coord)

    for name, func in [
        ("clean_degree_with_comma", clean_degree_with_comma),
        ("clean_degree_separated", clean_degree_separated),
        ("clean_degree_as_separator", clean_degree_as_separator),
        ("clean_two_commas_with_space", clean_two_commas_with_space),
        ("clean_comma_separated", clean_comma_separated),
        ("clean_merged_coordinates", clean_merged_coordinates),
        ("move_comma_separator", move_comma_separator),
        ("clean_dot_separated_no_comma", clean_dot_separated_no_comma),
        ("move_comma_separator", move_comma_separator)
    ]:
        result = func(coord)
        if result:
            return result, name
    return coord, "original (no match)"

# ==================== Data Uji dan Debugging ====================
raw_data = [
    "-7.361902Â,112.693948Â",     # Harus jadi: -7.361902,112.693948
    "-7.361902Â°112.693948Â°",     # Harus jadi: -7.361902,112.693948
    "-6.9271° 107.6048°",          # Harus jadi: -6.9271,107.6048
    "-8,1948403, 111,1077904",     # Harus jadi: -8.1948403,111.1077904
    "8.180339,111.116929",         # Sudah bersih
    "-7,2892906 112,7276532",      # Harus jadi: -7.2892906,112.7276532
    "-7.36271456342.732918",       # Harus jadi: -7.362712,112.732918
    "-7.1757799948114.112.60744746700044",        # Harus jadi: -7.362712,112.732918
]

# ==================== Output Hasil ====================
for i, coord in enumerate(raw_data):
    cleaned, method = apply_cleaning_debug(coord)
    print(f"{i+1}. Original: {coord}")
    print(f"   ✅ Cleaned : {cleaned}")
    print(f"   🔍 Method  : {method}\n")


1. Original: -7.361902Â,112.693948Â
   ✅ Cleaned : -7.361902,112.693948
   🔍 Method  : clean_degree_with_comma

2. Original: -7.361902Â°112.693948Â°
   ✅ Cleaned : -7.361902,112.693948
   🔍 Method  : clean_degree_separated

3. Original: -6.9271° 107.6048°
   ✅ Cleaned : -6.9271, 107.6048
   🔍 Method  : clean_degree_as_separator

4. Original: -8,1948403, 111,1077904
   ✅ Cleaned : -8,1948403, 111,1077904
   🔍 Method  : original (no match)

5. Original: 8.180339,111.116929
   ✅ Cleaned : 8.180339,111.116929
   🔍 Method  : clean_comma_separated

6. Original: -7,2892906 112,7276532
   ✅ Cleaned : -7.2892906,112.7276532
   🔍 Method  : clean_two_commas_with_space

7. Original: -7.36271456342.732918
   ✅ Cleaned : -7.36271456,342.732918
   🔍 Method  : clean_merged_coordinates

8. Original: -7.1757799948114.112.60744746700044
   ✅ Cleaned : -7.1757799948114,112.60744746700044
   🔍 Method  : clean_dot_separated_no_comma



In [43]:
# ========================= Cleansing Kolom latitude_olt =========================
olt_data = aset_data[[
    "Hostname OLT", "Latitude OLT","Longtitude OLT", "Brand OLT", "Type OLT",
    "Kapasitas OLT", "Kapasitas port OLT", "OLT Port", "OLT", "Interface OLT"
]].reset_index(drop=True)

olt_data.columns = [
    "hostname_olt", "latitude_olt","longitude_olt", "brand_olt", "type_olt",
    "kapasitas_olt", "kapasitas_port_olt", "olt_port", "olt", "interface_olt"
]
olt_data['latitude_olt'] = olt_data['latitude_olt'].apply(apply_cleaning)
olt_data[["latitude_olt_clean", "longitude_olt_clean"]] = olt_data["latitude_olt"].str.split(",", expand=True)

# Jika longitude_olt kosong (NaN), gunakan nilai dari latitude_olt_clean
olt_data["longitude_olt"] = olt_data["longitude_olt"].combine_first(olt_data["latitude_olt_clean"])

olt_data.drop(columns=["latitude_olt_clean", "longitude_olt_clean"], inplace=True)
# Output informasi DataFrame untuk memastikan hasil
olt_data.info()

# ========================= Simpan Hasil =========================
aset_data.to_csv("data_clean/OLT.csv", index=False)

print(f"\n✅ Berhasil save file ke data_clean.")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30818 entries, 0 to 30817
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   hostname_olt        30759 non-null  string
 1   latitude_olt        30627 non-null  object
 2   longitude_olt       30627 non-null  object
 3   brand_olt           30601 non-null  string
 4   type_olt            30490 non-null  string
 5   kapasitas_olt       28981 non-null  Int64 
 6   kapasitas_port_olt  19173 non-null  Int64 
 7   olt_port            28250 non-null  Int64 
 8   olt                 30802 non-null  string
 9   interface_olt       3556 non-null   string
dtypes: Int64(3), object(2), string(5)
memory usage: 2.4+ MB

✅ Berhasil save file ke data_clean.


In [53]:
# Tabel FDT
fdt_data = aset_data[[
    "FDTID", "Status OSP AMARTA FDT", "Jumlah Splitter FDT", "Kapasitas Splitter FDT",
    "FDT New/Existing", "Port FDT", "Latitude FDT", "Longtitude FDT"
]].reset_index(drop=True)

fdt_data.columns = [
    "fdt_id", "status_osp_amarta_fdt","jumlah_splitter_fdt", "kapasitas_splitter_fdt", "fdt_new_existing",
    "port_fdt", "latitude_fdt", "longitude_fdt"
]

fdt_data['latitude_fdt'] = fdt_data['latitude_fdt'].apply(apply_cleaning)

fdt_data[["latitude_fdt_clean", "longitude_fdt_clean"]] = fdt_data["latitude_fdt"].str.split(",", n=1, expand=True)

# Jika longitude_olt kosong (NaN), gunakan nilai dari latitude_olt_clean
fdt_data["longitude_fdt"] = fdt_data["longitude_fdt"].combine_first(fdt_data["latitude_fdt_clean"])

fdt_data.drop(columns=["latitude_fdt_clean", "longitude_fdt_clean"], inplace=True)
# Output informasi DataFrame untuk memastikan hasil
fdt_data.info()

fdt_data.to_csv("FDT.csv", index=False)
print(f"\n✅ Berhasil save file ke data_clean.")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30818 entries, 0 to 30817
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   fdt_id                  30617 non-null  string
 1   status_osp_amarta_fdt   96 non-null     string
 2   jumlah_splitter_fdt     22597 non-null  Int64 
 3   kapasitas_splitter_fdt  28360 non-null  Int64 
 4   fdt_new_existing        18698 non-null  string
 5   port_fdt                21357 non-null  Int64 
 6   latitude_fdt            30601 non-null  object
 7   longitude_fdt           30601 non-null  object
dtypes: Int64(3), object(2), string(3)
memory usage: 2.0+ MB

✅ Berhasil save file ke data_clean.


In [10]:
fdt_data["latitude_fdt"].head(10)

,latitude_fdt
0,-7.4621
1,-7.4621
2,-7.4621
3,-7.4621
4,-7.4621
5,-7.4621
6,-7.4621
7,-7.4621
8,-7.4621
9,-7.4621


In [ ]:
# Tabel FAT
fat_data = aset_data[[
    "FATID", "Jumlah Splitter FAT", "Kapasitas Splitter FAT", "Latitude FAT", "Longtitude FAT",
    "FDTID", "Status OSP AMARTA FAT", "FAT KONDISI", "FAT FILTER PEMAKAIAN",
    "KETERANGAN FULL", "FAT ID X", "FILTER FAT CAP"
]].copy()

fat_data = fat_data.rename(columns={
    "FATID": "fatid",
    "Jumlah Splitter FAT": "jumlah_splitter_fat",
    "Kapasitas Splitter FAT": "kapasitas_splitter_fat",
    "Latitude FAT": "latitude_fat",
    "Longtitude FAT": "longitude_fat",
    "FDTID": "fdt_id",
    "Status OSP AMARTA FAT": "status_osp_amarta_fat",
    "FAT KONDISI": "fat_kondisi",
    "FAT FILTER PEMAKAIAN": "fat_filter_pemakaian",
    "KETERANGAN FULL": "keterangan_full",
    "FAT ID X": "fatid_x",
    "FILTER FAT CAP": "filter_fat_cap"
})
fat_data.to_csv("FAT.csv", index=False)

# Tabel Cluster
cluster_data = aset_data[[
    "Latitude Cluster", "Longtitude Cluster", "Area KP", "Kota Kab", "Kecamatan",
    "Kelurahan", "UP3", "ULP", "FATID"
]].copy()

cluster_data = cluster_data.rename(columns={
    "Latitude Cluster": "latitude_cluster",
    "Longtitude Cluster": "longitude_cluster",
    "Area KP": "area_kp",
    "Kota Kab": "kota_kab",
    "Kecamatan": "kecamatan",
    "Kelurahan": "kelurahan",
    "UP3": "up3",
    "ULP": "ulp",
    "FATID": "fatid"
})
cluster_data["id_cluster"] = cluster_data.index + 1
cluster_data.to_csv("Cluster.csv", index=False)

# Tabel Dokumentasi
dokumentasi_data = aset_data[[
    "Status OSP AMARTA FAT", "LINK DOKUMEN FEEDER", "KETERANGAN DOKUMEN",
    "LINK DATA ASET", "KETERANGAN DATA ASET", "LINK MAPS",
    "UPDATE ASET", "AMARTA UPDATE", "FATID"
]].copy()

dokumentasi_data = dokumentasi_data.rename(columns={
    "Status OSP AMARTA FAT": "status_osp_amarta_fat",
    "LINK DOKUMEN FEEDER": "link_dokumen_feeder",
    "KETERANGAN DOKUMEN": "keterangan_dokumen",
    "LINK DATA ASET": "link_data_aset",
    "KETERANGAN DATA ASET": "keterangan_data_aset",
    "LINK MAPS": "link_maps",
    "UPDATE ASET": "update_aset",
    "AMARTA UPDATE": "amarta_update",
    "FATID": "fatid"
})
dokumentasi_data["id_dokumentasi"] = dokumentasi_data.index + 1
dokumentasi_data.to_csv("Dokumentasi.csv", index=False)

# Tabel HomeConnected
homeconnected_data = aset_data[[
    "HC OLD", "HC iCRM+", "TOTAL HC", "CLEANSING HP", "FATID"
]].copy()

homeconnected_data = homeconnected_data.rename(columns={
    "HC OLD": "hc_old",
    "HC iCRM+": "hc_icrm",
    "TOTAL HC": "total_hc",
    "CLEANSING HP": "cleansing_hp",
    "FATID": "fatid"
})
homeconnected_data["id_homeconnected"] = homeconnected_data.index + 1
homeconnected_data.to_csv("HomeConnected.csv", index=False)

# Tabel AdditionalInformation
additional_info_data = aset_data[[
    "PA", "Tanggal RFS", "Mitra", "Kategori", "Sumber Datek", "FATID"
]].copy()

additional_info_data = additional_info_data.rename(columns={
    "PA": "pa",
    "Tanggal RFS": "tanggal_rfs",
    "Mitra": "mitra",
    "Kategori": "kategori",
    "Sumber Datek": "sumber_datek",
    "FATID": "fatid"
})
additional_info_data["id_additionalinfo"] = additional_info_data.index + 1
additional_info_data.to_csv("AdditionalInformation.csv", index=False)

# ========================= Data USER =========================
# Tabel Pelanggan
pelanggan_data = user_data[[
    "ID Permohonan", "SID", "Cust Name", "Telpn",
    "Latitude Pelanggan", "Longtitude Pelanggan", "ID FAT", "NOTES"
]].copy()

pelanggan_data = pelanggan_data.rename(columns={
    "ID Permohonan": "id_permohonan",
    "SID": "sid",
    "Cust Name": "cust_name",
    "Telpn": "telpn",
    "Latitude Pelanggan": "latitude_pelanggan",
    "Longtitude Pelanggan": "longitude_pelanggan",
    "ID FAT": "fatid",
    "NOTES": "notes"
})
pelanggan_data.to_csv("Pelanggan.csv", index=False)

print("✅ Semua tabel berhasil disesuaikan dengan struktur SQL dan disimpan sebagai CSV.")



In [3]:
import pandas as pd
import re

def split_lat_lon(bad_value):
    if pd.isna(bad_value):
        return None, None

    # Cari dua titik dalam string
    parts = [m.start() for m in re.finditer(r'\.', str(bad_value))]

    if len(parts) < 2:
        return None, None

    second_dot = parts[1]
    lon_start = second_dot - 3

    try:
        lat = float(bad_value[:lon_start])
        lon = float(bad_value[lon_start:])
        return lat, lon
    except ValueError:
        return None, None

# Path file
input_file = "/content/OLT.csv"
output_file = "/content/OLT_cleaned.csv"

# Baca file
df = pd.read_csv(input_file)

# Proses hanya kolom 'latitude_olt' → pisahkan jadi dua
latitudes = []
longitudes = []

for value in df['latitude_olt']:
    lat, lon = split_lat_lon(value)
    latitudes.append(lat)
    longitudes.append(lon)

# Overwrite nilai kolom
df['latitude_olt'] = latitudes
df['longitude_olt'] = longitudes  # Ini overwrite kolom yang sudah ada

# Simpan hasil
df.to_csv(output_file, index=False)
print(f"Data berhasil dipisah dan disimpan ke: {output_file}")


Data berhasil dipisah dan disimpan ke: /content/OLT_cleaned.csv
